# Exploring owner_1 Truncation
In this notebook I will be attempting to cross reference property assessment data with Business License data

In [1]:
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
import csv
import re
from fuzzywuzzy import fuzz

### Putting Business License data into smaller JSON
Only grabbing what I need at first from the CSV

In [13]:
def get_useful_li_data(source, output):
    data = {}
    line_count = 0
    with open(source, mode="r") as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in tqdm(csv_reader, total=366036):
            if line_count == 0:
                line_count += 1
            else:
                if row["address"].strip() not in data.keys():
                    data[row["address"].strip()] = {
                        'total_address_entries': 1,
                        'entries': {
                            row["legalname"].strip(): {
                                'license_num': row['licensenum'].strip(),
                                'license_type': row['licensetype'].strip(),
                                'initial_issue_date': row['initialissuedate'].strip(),
                                'most_recent_issue_date': row['mostrecentissuedate'].strip(),
                                'expiration_date': row['expirationdate'].strip(),
                                'inactive_date': row['inactivedate'].strip(),
                                'license_status': row['licensestatus'].strip(),
                                'parcel_id_num': row['parcel_id_num'].strip(),
                                'opa_account_num': row['opa_account_num'].strip(),
                                'opa_owner': row['opa_owner'].strip(),
                                'number_of_units': row['numberofunits'].strip(),
                                'legal_entity_type': row['legalentitytype'].strip(),
                                'business_name': row['business_name'].strip(),
                                'business_mailing_address': row['business_mailing_address'].strip()
                            }
                        }
                    }
                else:
                    # Setting the data to var names for easier reading
                    address = data[row['address'].strip()]
                    legal_name = row["legalname"].strip()
                    license_num = row['licensenum'].strip()
                    license_type = row['licensetype'].strip()
                    initial_issue_date = row['initialissuedate'].strip()
                    most_recent_issue_date = row['mostrecentissuedate'].strip()
                    expiration_date = row['expirationdate'].strip()
                    inactive_date = row['inactivedate'].strip()
                    license_status = row['licensestatus'].strip()
                    parcel_id_num = row['parcel_id_num'].strip()
                    opa_account_num = row['opa_account_num'].strip()
                    opa_owner = row['opa_owner'].strip()
                    number_of_units = row['numberofunits'].strip()
                    legal_entity_type =  row['legalentitytype'].strip()
                    business_name = row['business_name'].strip()
                    business_mailing_address = row['business_mailing_address'].strip()

                    # Update and add values
                    address['total_address_entries'] += 1
                    address['entries'][legal_name] = {
                                'license_num': license_num,
                                'license_type': license_type,
                                'initial_issue_date': initial_issue_date,
                                'most_recent_issue_date': most_recent_issue_date,
                                'expiration_date': expiration_date,
                                'inactive_date': inactive_date,
                                'license_status': license_status,
                                'parcel_id_num': parcel_id_num,
                                'opa_account_num': opa_account_num,
                                'opa_owner': opa_owner,
                                'number_of_units': number_of_units,
                                'legal_entity_type': legal_entity_type,
                                'business_name': business_name,
                                'business_mailing_address': business_mailing_address
                            }
                line_count += 1
        print('There are ', line_count, 'in this dataset.')
        with open(output, 'w+') as file:
            file.write(json.dumps(data))         

In [15]:
get_useful_li_data('./../../data_sets/business_licenses.csv', './data/business_licenses.json')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 366036/366036 [00:04<00:00, 74558.76it/s]


There are  366036 in this dataset.


### Merging Business License Data with Property Assessment Data

In [30]:
def merge_PA_BL(PA_source, BL_source, output):
    PA_data = json.load(open(PA_source))
    BL_data = json.load(open(BL_source))
    
    count = 0
    
    for owner in tqdm(PA_data, total=len(PA_data)):
        properties = PA_data[owner]['properties']
        for address in properties:
            if address in BL_data.keys():
                PA_data[owner]['properties'][address]['business_license_entries'] = BL_data[address]
                PA_data[owner]['properties'][address]['has_business_license_entry'] = True
                count += 1
            else:
                PA_data[owner]['properties'][address]['business_license_entries'] = []
                PA_data[owner]['properties'][address]['has_business_license_entry'] = False                            
    json.dump(PA_data, open(output, 'w+'))
    
    print('There were ', count, ' entries linked.')

In [29]:
merge_PA_BL('./../../data_sets/landlords_and_properties.json', './data/business_licenses.json', './data/PA_BL_landlords_and_properties.JSON')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 429982/429982 [00:00<00:00, 545912.76it/s]


There were  195341  entries linked.


### Verifing the Previous Function
I need to split the JSON into 2 because it is too large to open. Splitting it by having an match and not.

In [38]:
def split_PA_BL_results(source, match_output):
    data = json.load(open(source))
    output = {}
    split_count = 0
    for owner in data:
        entry_flag_count = 0
        total_props = data[owner]['total_properties']
        bl_props = {}
        props = {}
        for address in data[owner]['properties']:
            if data[owner]['properties'][address]['has_business_license_entry'] is True:
                entry_flag_count += 1
                bl_props[address] = data[owner]['properties'][address]
            else:
                props[address] = data[owner]['properties'][address]
        if entry_flag_count > 0:
            split_count += 1
            output[owner] = {
                'total_properties': total_props,
                'bl_property_count': entry_flag_count,
                'business_license_properties': bl_props,
                'other_properties': props
            }
    json.dump(output, open(match_output, 'w+'))
    print('A total of ', split_count, 'things were split.')

In [39]:
split_PA_BL_results('./data/PA_BL_landlords_and_properties.JSON', './data/split_landlords_and_properties.JSON')

Exception ignored in: <function tqdm.__del__ at 0x000001FB4F148820>
Traceback (most recent call last):
  File "c:\python38\lib\site-packages\tqdm\std.py", line 1090, in __del__
    self.close()
  File "c:\python38\lib\site-packages\tqdm\std.py", line 1303, in close
    self.display(pos=0)
  File "c:\python38\lib\site-packages\tqdm\std.py", line 1481, in display
    self.sp(self.__repr__() if msg is None else msg)
  File "c:\python38\lib\site-packages\tqdm\std.py", line 1093, in __repr__
    return self.format_meter(**self.format_dict)
  File "c:\python38\lib\site-packages\tqdm\std.py", line 425, in format_meter
    bool_prefix_colon_already = (prefix[-2:] == ": ")
TypeError: 'int' object is not subscriptable


A total of  134595 things were split.


### Getting 'L' possibilities
Find all the owners with 'L' at the end of the truncated name.

In [2]:
def get_L_posibilities(source, output):
    data = json.load(open(source))
    out = []
    count = 0
    property_count = 0
    L_count = 0
    
    raw_search_string_L = r"\b" + "L" + r"\b$"
    
    for owner in tqdm(data, total=len(data)):
        match_output_L = re.search(raw_search_string_L, owner[0])
        if match_output_L is not None:
            out.append(owner)
            count += 1
            property_count += owner[1]
            if match_output_L is not None:
                L_count += 1

    json.dump(out, open(output, 'w+'))
    print('There are ', L_count, 'Ls in this dataset')

In [3]:
get_L_posibilities('./../../data_sets/sorted_landlords.json', './data/L_owner.json')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 429983/429983 [00:00<00:00, 992858.28it/s]


There are  10594 Ls in this dataset


### Looking for L's in the matched Business license owners

In [2]:
def match_L_possiblities(L_source, BL_source, match_output, no_match_output):
        L_data = json.load(open(L_source))
        BL_data = json.load(open(BL_source))
        match_o = {}
        not_match_o = {}
        count = 0
        not_count = 0
        for L_owner in tqdm(L_data, total=len(L_data)):
            owner_name = L_owner[0]
            for BL_owner in BL_data:
                if BL_owner == owner_name and BL_data[BL_owner]['bl_property_count'] > 0:
                    match_o[owner_name] = BL_data[BL_owner]
                    count += 1
                else:
                    not_match_o[owner_name] = BL_data[BL_owner]
                    not_count += 1
        json.dump(match_o, open(match_output, 'w+'))
        json.dump(not_match_o, open(no_match_output, 'w+'))
        print('There were ', count, 'matches.')
        print('There were ', not_count, 'not matches.')
                    

In [3]:
match_L_possiblities('./data/L_owner.json', './data/split_landlords_and_properties.JSON', './data/L_split_match.JSON', './data/L_split_no_match.JSON')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10594/10594 [07:13<00:00, 24.42it/s]


There were  2847 matches.
There were  1425896583 not matches.


### Finding L's with LLC Business License Entries

In [38]:
def get_Ls_with_LLC_BL_entries(source, match_output, no_match_owner_output):
    data = json.load(open(source))
    count = 0
    out = {}
    raw_search_string_LLC = r"\b" + "LLC" + r"\b"
    no_match_owners = []
    no_match_count = 0
    for owner in tqdm(data, total=len(data)):
        match_count = 0
        matches = {}
        bl_props = data[owner]['business_license_properties']
        for prop in bl_props:
            entries = bl_props[prop]['business_license_entries']['entries']
            entry_matches = {}
            for entry in entries:
                match_output_LLC_entry = re.search(raw_search_string_LLC, entry)
                match_output_LLC_business_name = re.search(raw_search_string_LLC, entries[entry]['business_name'])
                if match_output_LLC_entry is not None or match_output_LLC_business_name is not None:
                    match_count += 1
                    entry_matches[entry] = entries[entry]
            if entry_matches != {}:
                matches[prop] = entry_matches
        if match_count > 0:
            out[owner] = matches
            count += 1
        else:
            no_match_count += 1
            no_match_owners.append(owner)
            
    json.dump(out, open(match_output, 'w+'))
    json.dump(no_match_owners, open(no_match_owner_output, 'w+'))
    print('There were ', count, 'matches.')
    print('There were ', no_match_count, 'not matches.')

In [39]:
get_Ls_with_LLC_BL_entries('./data/L_split_match.JSON', './data/L_split_LLC_match.JSON', './data/L_split_no_LLC_match.JSON')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2847/2847 [00:00<00:00, 33887.04it/s]

There were  625 matches.
There were  2222 not matches.


### Creating Fuzzy String Match Ranking to find actual matches
I need to iterate over each PA owner and compare them to entries in each matched BL property entry

In [10]:
def fuzzy_match_owners(source, output, distance_lower_bound):
    data = json.load(open(source))
    out = {}
    possible_matches_count = 0
    entries_to_check_count = 0
    for owner in tqdm(data, total=len(data)):
        fuzzy_ratios = {}
        for address in data[owner]:
            for entry in data[owner][address]:
                entry_owner_object = data[owner][address][entry]
                entry_business_name = data[owner][address][entry]['business_name']
                license_type = data[owner][address][entry]['license_type']
                legal_entity_type = data[owner][address][entry]['legal_entity_type']
                license_status = data[owner][address][entry]['license_status']
                business_mailing_address = data[owner][address][entry]['business_mailing_address']
                
                is_rental = False
                is_active_license = False
                if license_type == "Rental":
                    is_rental = True
                if license_status == "Active":
                    is_active_license = True
                
                
                entry_owner_fuzz_ratio = fuzz.ratio(owner, entry)
                entry_business_name_fuzz_ratio = fuzz.ratio(owner, entry_business_name)
                
                if entry_owner_fuzz_ratio >= distance_lower_bound or entry_business_name_fuzz_ratio >= distance_lower_bound:
                    entries_to_check_count += 1
                    fuzzy_ratios[entry] = {
                        'entry_owner_fuzz_ratio': entry_owner_fuzz_ratio,
                        'business_name': entry_business_name,
                        'business_name_fuzz_ratio': entry_business_name_fuzz_ratio,
                        'is_rental': is_rental,
                        'is_active_license':is_active_license,
                        'legal_entity_type': legal_entity_type,
                        'address': address
                    }
        if fuzzy_ratios != {}:
            possible_matches_count += 1
            out[owner] = fuzzy_ratios
    json.dump(out, open(output, 'w+'))
    print('There were ', possible_matches_count, 'possible matches with a score >= ', distance_lower_bound, '.')
    print("That's ", entries_to_check_count, 'entries to check by hand! Wooohoooo!')

In [13]:
fuzzy_match_owners('./data/L_split_LLC_match.JSON', './data/fuzz_ratio_L_split_LLC_match.JSON', 70)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:00<00:00, 89270.90it/s]

There were  201 possible matches with a score >=  70 .
That's  415 entries to check by hand! Wooohoooo!
